# The Dutch National Flag Problem

The quicksort algorithm for sorting arrays proceeds recursively - it selects an 
element (the "pivot"), reorders the array to make all the elements less than or
equal to the pivot appear first, followed by all the elements greater than the pivot.
The two subarrays are then sorted recursively.

Implemented naively, quicksort has large run times and deep function call stacks on
arrays with many duplicates because the subarrays may differ greatly in size.  One
solution is to reorder the array so that all elements less than the pivot appear
first, followed by elements equal to the pivot, followed by elements greater than
the pivot.  This is known as Dutch National Flag partitioning, because the Dutch
national flag consists of three horizontal bands, each in a different color.

For example, suppose `A = [0,1,2,0,2,1,1]`, and the pivot index is `3`.  Then 
`A[3] = 0`, so `[0,0,1,2,2,1,1]` is a valid partitioning.  For the same array, if
the pivot index is `2`, then `A[2] = 2`, so the arrays `[0,1,0,1,1,2,2]` as wells as
`[0,0,1,1,1,2,2]` are valid partitionings.

**Write a program that takes an array `A` and an index `i` into `A`, and
rearranges the elements such that all elements less than `A[i]` (the "pivot")
 appear first, followed by elements equal to the pivot, followed by elements
 greater than the pivot.**
 
 ## Brute Force Method
 
 The Problem is trivial to solve with `O(n)` additional space, where `n` is the
 length of `A`.  We form three lists, namely, elements less than the pivot, elements
 equal to the pivot, and elements greater than the pivot.  Consequently, we write 
 these values into `A`.  The time complexity is `O(n)`.
 
 We can avoid using `O(n)` additional space at the cost of increased time complexity
 as follows.  In the first stage, we iterate through `A` starting from index 0, then
 index 1, etc.  In each iteration, we seek an element smaller than the pivot - as
 soon as we find it, we move it to the subarray of smaller elements via an exchange.
 This moves all the elements less than the pivot to the start of the array.  The 
 second stage is similar to the first one, the difference being that we move elements
 greater than the pivot to the end of the array: 
 

In [17]:
RED, WHITE, BLUE = range(3)

unsorted_array = [0,1,0,2,0,1,1,2,2]
print("Unsorted Array:")
print(unsorted_array)

def dutch_flag_partition(pivot_index, A):
    pivot = A[pivot_index]
    # First pass: group elements smaller than the pivot
    for i in range(len(A)):
        # Look for a smaller element.
        for j in range(i + 1, len(A)):
            if A[j] < pivot:
                A[i], A[j] = A[j], A[i]
                break
    # Second pass: group elements larger than the pivot.
    for i in reversed(range(len(A))):
        if A[i] < pivot:
            break
        # Look for a larger element.  Stop when we reach an element less than
        # pivot, since first pass has moved then to the start of A.
        for j in reversed(range(i)):
            if A[j] > pivot:
                A[i], A[j] = A[j], A[i]
                break
                
dutch_flag_partition(WHITE, unsorted_array)
print("Sorting the Array using the median value (WHITE):")
print(unsorted_array)

Unsorted Array:
[0, 1, 0, 2, 0, 1, 1, 2, 2]
Sorting the Array using the median value (WHITE):
[0, 0, 0, 1, 1, 1, 2, 2, 2]


The additional space complexity is now `O(1)`, but the time complexity is
*O(n<sup>2</sup>)*, eg, if `i = n / 2` and all elements before `i` are greater
than `A[i]`, and all elements after `i` are less than `A[i]`.  Intuitively,
this approach has bad time complexity because in the first pass when searching
for each additional element smaller than the pivot we start from the beginning.
However, there is no reason to start from so far back - we can begin from the last
location we advanced to.  Same goes for the second pass.

## Improving Performance

To improve time complexity, we make a single pass and move all the elements less
than the pivot to the beginning.  In the second pass we move the larger elements
to the end.  It is easy to perform each pass in a single iteration, moving
out-of-place elements as soon as they are discovered.

In [22]:
unsorted_array = [0,1,0,2,0,1,1,2,2]
print("Unsorted Array:")
print(unsorted_array)

def dutch_flag_partition(pivot_index, A):
    pivot = A[pivot_index]
    # First pass: group elements smaller than the pivot.
    smaller = 0
    for i in range(len(A)):
        if A[i] < pivot:
            A[i], A[smaller] = A[smaller], A[i]
            smaller += 1
    # Second pass: group elements larger than the pivot.
    larger = len(A) - 1
    for i in reversed(range(len(A))):
        if A[i] < pivot:
            break
        elif A[i] > pivot:
            A[i], A[larger] = A[larger], A[i]
            larger -= 1

dutch_flag_partition(WHITE, unsorted_array)
print("Sorting the Array using the median value (WHITE):")
print(unsorted_array)

Unsorted Array:
[0, 1, 0, 2, 0, 1, 1, 2, 2]
Sorting the Array using the median value (WHITE):
[0, 0, 0, 1, 1, 1, 2, 2, 2]


The time complexity is `O(n)` and the space complexity is `O(1)`.

## Sorting in a Single Pass

This algorithm is similar to the one above, but it performs classification
into elements less than, equal to, and greater than the pivot in a single pass.
This reduces runtime, at the cost of a trickier implementation.  We do this by
maintaining four subarrays: *bottom, middle, unclassified, and top*.  Initially,
all elements are unclassified.  We iterate through elements in unclassified, and 
move elements into on of bottom, middle and top groups according to the relative
order between incoming unclassified elements and the pivot.

As a concrete example, suppose the array is currently `A = [-3,0,-1,1,1,?,?,?,4,2]`,
where the pivot is 1 and ? denotes unclassified elements.  There are three
possibilities for the first unclassified element, `A[5]`.
 
- `A[5]` is less than the pivot, eg `A[5] = -5`.  We exchange it with the first 1,
   ie the new array is `[-3,0,-1,-5,1,1,?,?,4,2]`.
- `A[5]` is equal to the pivot, ie `A[5] = 1`.  We do not need to move it, we just
   advance to the next unclassified element, ie the array is 
   `[-3,0,-1,1,1,1,?,?,4,2]`.
- `A[5]` is greater than the pivot, eg `A[5] = 3`.  We exchange it with the last
   unclassified element, ie the new array is `[-3,0,-1,1,1,?,>,3,4,2]`.
   

In [23]:
unsorted_array = [0,1,0,2,0,1,1,2,2]
print("Unsorted Array:")
print(unsorted_array)

def dutch_flag_partition(pivot_index, A):
    pivot = A[pivot_index]
    """
    Keep the Following INvarants During Partitioning:
    bottom group: A[:smaller].
    middle group: A[smaller:equal].
    unclassified group: A[equal:larger].
    top group: A[larger:].
    """
    smaller, equal, larger = 0, 0, len(A)
    # Keep iterating as long as there is an unclassified element.
    while equal < larger:
        # A[equal] is the incoming unclassified element.
        if A[equal] < pivot:
            A[smaller], A[equal] = A[equal], A[smaller]
            smaller, equal, = smaller + 1, equal + 1
        elif A[equal] == pivot:
            equal += 1
        else:   # A[equal] > pivot.
            larger -= 1
            A[equal], A[larger] = A[larger], A[equal]

dutch_flag_partition(WHITE, unsorted_array)
print("Sorting the Array using the median value (WHITE):")
print(unsorted_array)

Unsorted Array:
[0, 1, 0, 2, 0, 1, 1, 2, 2]
Sorting the Array using the median value (WHITE):
[0, 0, 0, 1, 1, 1, 2, 2, 2]


Each iteration decreases the size of unclassified by 1, and the time spent within
each iteration is `O(1)`, implying the time complexity is `O(n)`.  The space
complexity is `O(1)`.
